# b) Ngonye Falls Synthetic Historic Flow Series

To produce a long time series (1924-2017) of daily flow for the Ngonye Falls site, a correlation is carried out between the overlapping portions of the *Zambezi River Authority* (ZRA) gauge at Ngonye Falls and portion of the record of the Victoria Falls gauge (2005-2016).

Once correlated, the full Victoria Falls gauge record is extrapolated to produce a synthetic record for Ngonye Falls.

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

1. Calculate Flow Duration Curves (FDCs) from daily data for the following:
    1. The target site (Ngonye) for 2005/06 – 2016/17; 
    2. The analogue site (Victoria Falls) for the exact period of data at the target site (2005/06 – 2016/17);
    3. Analogue site for long-term record (1924/25 -2016/17); 
    
    
2. Compare the FDCs for the analogue site for the two periods and derive factors between the shorter and longer periods; 

3. Apply these factors to the target site FDC to produce an estimated long-term FDC for the target site;

4. For each daily flow value at the analogue site, determine its position on the long-term FDC (percentile); and,

5. Look up the flow value for this percentile from the estimated long-term FDC for the target site taking into account of the 11-day time lag.

*Water Year* is defined as running Oct to Sept as is usual in the region.

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| daily_gauge_vicfalls.csv        | Zambezi River Authority | Daily Flow for Victoria Falls 1924/25 - 2016/17              |
| daily_gauge_ngonye.csv | Zambezi River Authority   | Daily Stage (Level) for Ngonye Falls 2005/06 – 2016/17 |
|  Stage - Discharge equation for Ngonye Falls |  Mott MacDonald |Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D |


## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_synthetic.csv  | Synthetic daily flow series for Ngonye  Falls 1924/25 - 2016/17  |
| ngonye_vicfalls_fdcs.csv           | Flow Duration Curves for Victoria Falls and Ngonye Falls |


## Libraries

In [21]:
import numpy as np
import pandas as pd
import datetime

## Parameters

In [22]:
input_data='./input_data/2022/'
output_data='./output_data/2022/'
#overlap_cutoff=datetime.datetime(2017, 9,30)
overlap_cutoff=datetime.datetime(2022, 10,11)

## Load Data
Load the Vic Falls data and add some helper columns.

In [23]:
vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls_2022.csv")
#vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls.csv")

vicfalls['Date']=pd.to_datetime(vicfalls['Date'],format="%d/%m/%Y")
vicfalls=vicfalls.set_index(pd.DatetimeIndex(vicfalls['Date']))
vicfalls=vicfalls.drop(['Date'],axis=1)

vicfalls=vicfalls.astype({'Flow': 'float64'})
vicfalls['Flow']=np.round(vicfalls['Flow'],4)

vicfalls

,Flow
Date,
1924-10-01,100.0
1924-10-02,100.0
1924-10-03,100.0
1924-10-04,100.0
1924-10-05,100.0
...,...
2022-10-07,293.0
2022-10-08,289.0
2022-10-09,286.0


Load the Ngonye Falls level data and calculate flow based on the stage-discharge relationship:

\begin{equation*}
flow=1093.0355*(level-2.85)^{1.659}
\end{equation*}

Add some helper columns.

In [24]:
ngo=pd.read_csv(input_data+ "daily_gauge_ngonye_2022.csv")
#ngo=pd.read_csv(input_data+ "daily_gauge_ngonye.csv")
ngo['Date']=pd.to_datetime(ngo['Date'],format="%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo=ngo.drop(['Date'],axis=1)
ngo['Flow']=np.round(1093.0355*(ngo['Level']-2.85)**1.659,4)
ngo['VicFalls']=vicfalls['Flow']
ngo

,Level,Flow,VicFalls
Date,,,
2005-10-01,3.218,208.1497,211.8138
2005-10-02,3.216,206.2763,210.5129
2005-10-03,3.215,205.3422,208.5712
2005-10-04,3.213,203.4789,204.0855
2005-10-05,3.212,202.5498,204.0855
...,...,...,...
2022-09-26,3.296,286.3392,324.7189
2022-09-27,3.292,282.0914,322.5046
2022-09-28,3.292,282.0914,318.0918


## Flow Duration Curve

Build the Flow Duration Curve table in 0.1% exceedance increments.

Add FDC flows for the full Vic Falls timeseries.

In [25]:
fdc=pd.DataFrame({'Exceedance': np.arange(0,1.001,0.001)}).set_index('Exceedance')
fdc['VicFalls_full']=np.round(np.quantile(vicfalls.loc[vicfalls.index<=overlap_cutoff]['Flow'],1-fdc.index),4)
fdc

,VicFalls_full
Exceedance,
0.000,9435.5576
0.001,8391.2178
0.002,7424.0498
0.003,6653.5660
0.004,6171.5670
...,...
0.996,124.7354
0.997,115.2667
0.998,107.1446


Add flows for Ngonye to the FDC.

Add flows for the portion of the Vic Falls series that overlaps with the Ngonye Series.

In [26]:
fdc['Ngonye_gauged']=np.round(np.percentile(ngo.loc[ngo.index<=overlap_cutoff]['Flow'],((1-fdc.index)*100)),4)
fdc['VicFalls_overlap']=np.round(np.percentile(ngo.loc[ngo.index<=overlap_cutoff]['VicFalls'],((1-fdc.index)*100)),4)
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap
Exceedance,,,
0.000,9435.5576,5935.9208,5650.5399
0.001,8391.2178,5782.2643,5561.7039
0.002,7424.0498,5595.7437,5355.4878
0.003,6653.5660,5434.6644,5254.8841
0.004,6171.5670,5118.4467,4975.7204
...,...,...,...
0.996,124.7354,154.9316,144.0799
0.997,115.2667,153.2661,135.6593
0.998,107.1446,152.4361,121.0092


## Vic Falls Ratio
Calculate the ratio of Vic Falls flows for the whole series and the overallping protion across the FDC.

Smooth that ratio (0.7% moving average) except at the tails of the FDC.

In [27]:
fdc['VicFalls_factor']=fdc['VicFalls_full']/fdc['VicFalls_overlap']
fdc['VicFalls_factor_smooth']=fdc['VicFalls_factor'].rolling(7,center=True).mean()
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name<0.01 else x['VicFalls_factor_smooth'])),axis=1)
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name>0.99 else x['VicFalls_factor_smooth'])),axis=1)
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth
Exceedance,,,,,
0.000,9435.5576,5935.9208,5650.5399,1.669851,1.669851
0.001,8391.2178,5782.2643,5561.7039,1.508749,1.508749
0.002,7424.0498,5595.7437,5355.4878,1.386251,1.386251
0.003,6653.5660,5434.6644,5254.8841,1.266168,1.266168
0.004,6171.5670,5118.4467,4975.7204,1.240336,1.240336
...,...,...,...,...,...
0.996,124.7354,154.9316,144.0799,0.865738,0.865738
0.997,115.2667,153.2661,135.6593,0.849678,0.849678
0.998,107.1446,152.4361,121.0092,0.885425,0.885425


## Scaled FDC
Produce a scaled FDC for Ngonye by using the factors calculated for the Vic Falls data.

In [28]:
fdc['Ngonye_scaled']=fdc['Ngonye_gauged']*fdc['VicFalls_factor_smooth']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled
Exceedance,,,,,,
0.000,9435.5576,5935.9208,5650.5399,1.669851,1.669851,9912.101075
0.001,8391.2178,5782.2643,5561.7039,1.508749,1.508749,8723.988186
0.002,7424.0498,5595.7437,5355.4878,1.386251,1.386251,7757.104758
0.003,6653.5660,5434.6644,5254.8841,1.266168,1.266168,6881.198060
0.004,6171.5670,5118.4467,4975.7204,1.240336,1.240336,6348.595621
...,...,...,...,...,...,...
0.996,124.7354,154.9316,144.0799,0.865738,0.865738,134.130126
0.997,115.2667,153.2661,135.6593,0.849678,0.849678,130.226808
0.998,107.1446,152.4361,121.0092,0.885425,0.885425,134.970770


## Conversion
Calculate a conversion factor for each row of the FDC between the scaled Ngonye FDC flows and the full Vic Falls series FDC.

In [29]:
fdc['Conversion']=fdc['Ngonye_scaled']/fdc['VicFalls_full']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion
Exceedance,,,,,,,
0.000,9435.5576,5935.9208,5650.5399,1.669851,1.669851,9912.101075,1.050505
0.001,8391.2178,5782.2643,5561.7039,1.508749,1.508749,8723.988186,1.039657
0.002,7424.0498,5595.7437,5355.4878,1.386251,1.386251,7757.104758,1.044862
0.003,6653.5660,5434.6644,5254.8841,1.266168,1.266168,6881.198060,1.034212
0.004,6171.5670,5118.4467,4975.7204,1.240336,1.240336,6348.595621,1.028685
...,...,...,...,...,...,...,...
0.996,124.7354,154.9316,144.0799,0.865738,0.865738,134.130126,1.075317
0.997,115.2667,153.2661,135.6593,0.849678,0.849678,130.226808,1.129787
0.998,107.1446,152.4361,121.0092,0.885425,0.885425,134.970770,1.259707


Lookup the conversion factors from the FDC based on flow and give each record in the full Vic Falls series its corresponding conversion factor. 

In [30]:
if 'Conversion' in vicfalls.columns:
    display(vicfalls.columns)
    vicfalls=vicfalls.drop(['Conversion','Exceedance'],axis=1)

#vicfalls['tmp_Flow']=100000-vicfalls['Flow']
#fdc['tmp_Flow']=100000-fdc['VicFalls_full']
fdc['ExceedanceInv']=1-fdc.index

tmp=pd.merge_asof(vicfalls.reset_index().sort_values('Flow',ascending=True),fdc.reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=True),left_on='Flow',right_on='VicFalls_full').set_index('Date')
vicfalls['Conversion']=tmp['Conversion']
vicfalls['Exceedance']=tmp['Exceedance']

vicfalls

,Flow,Conversion,Exceedance
Date,,,
1924-10-01,100.0,1.384095,0.999
1924-10-02,100.0,1.384095,0.999
1924-10-03,100.0,1.384095,0.999
1924-10-04,100.0,1.384095,0.999
1924-10-05,100.0,1.384095,0.999
...,...,...,...
2022-10-07,293.0,0.932126,0.831
2022-10-08,289.0,0.928947,0.836
2022-10-09,286.0,0.929724,0.841


## Ngonye Sythetic
Prepare the full  synthetic series for Ngonye by applying the 11 day lag to the Vic Falls series and the conversion factors calculated previoulsy.

In [31]:

ngonye_synth=pd.DataFrame(index=vicfalls.index)

ngonye_synth['LaggedDate']=ngonye_synth.index+pd.DateOffset(days=11)
ngonye_synth['VicFalls']=ngonye_synth.join(vicfalls,on='LaggedDate')['Flow']
ngonye_synth['Conversion']=ngonye_synth.join(vicfalls,on='LaggedDate')['Conversion']
ngonye_synth['Flow']=ngonye_synth['Conversion'] * ngonye_synth['VicFalls']

ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0,1.384095,138.409487
1924-10-02,1924-10-13,100.0,1.384095,138.409487
1924-10-03,1924-10-14,100.0,1.384095,138.409487
1924-10-04,1924-10-15,100.0,1.384095,138.409487
1924-10-05,1924-10-16,100.0,1.384095,138.409487
...,...,...,...,...
2022-10-07,2022-10-18,NaN,NaN,NaN
2022-10-08,2022-10-19,NaN,NaN,NaN
2022-10-09,2022-10-20,NaN,NaN,NaN


Remove extra columns and delete from the bottom to align to the water year.

In [32]:
ngonye_synth=ngonye_synth.dropna()
ngonye_synth=ngonye_synth.drop(ngonye_synth.loc['2022-10-01':].index)
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0,1.384095,138.409487
1924-10-02,1924-10-13,100.0,1.384095,138.409487
1924-10-03,1924-10-14,100.0,1.384095,138.409487
1924-10-04,1924-10-15,100.0,1.384095,138.409487
1924-10-05,1924-10-16,100.0,1.384095,138.409487
...,...,...,...,...
2022-09-26,2022-10-07,293.0,0.932126,273.112795
2022-09-27,2022-10-08,289.0,0.928947,268.465689
2022-09-28,2022-10-09,286.0,0.929724,265.901164


Add the new synthetic Ngonye flow series to the FDC.

In [33]:
fdc['Ngonye_synthetic']=np.percentile(ngonye_synth['Flow'],((1-fdc.index)*100))

tmp=pd.merge_asof(ngonye_synth.sort_values('Flow').reset_index(),fdc.sort_values('Ngonye_synthetic').reset_index(),left_on='Flow',right_on='Ngonye_synthetic').set_index('Date')
ngonye_synth['Exceedance']=tmp['Exceedance']

In [34]:
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow,Exceedance
Date,,,,,
1924-10-01,1924-10-12,100.0,1.384095,138.409487,0.998
1924-10-02,1924-10-13,100.0,1.384095,138.409487,0.998
1924-10-03,1924-10-14,100.0,1.384095,138.409487,0.998
1924-10-04,1924-10-15,100.0,1.384095,138.409487,0.998
1924-10-05,1924-10-16,100.0,1.384095,138.409487,0.998
...,...,...,...,...,...
2022-09-26,2022-10-07,293.0,0.932126,273.112795,0.825
2022-09-27,2022-10-08,289.0,0.928947,268.465689,0.839
2022-09-28,2022-10-09,286.0,0.929724,265.901164,0.847


# Prepare Export
Prepare a subset of the FDC for export

In [35]:
fdc_out=fdc.loc[:,['VicFalls_full','VicFalls_overlap','VicFalls_factor_smooth','Ngonye_gauged','Ngonye_scaled','Ngonye_synthetic','Conversion']]
fdc_out

,VicFalls_full,VicFalls_overlap,VicFalls_factor_smooth,Ngonye_gauged,Ngonye_scaled,Ngonye_synthetic,Conversion
Exceedance,,,,,,,
0.000,9435.5576,5650.5399,1.669851,5935.9208,9912.101075,9912.101075,1.050505
0.001,8391.2178,5561.7039,1.508749,5782.2643,8723.988186,8759.321492,1.039657
0.002,7424.0498,5355.4878,1.386251,5595.7437,7757.104758,7711.304088,1.044862
0.003,6653.5660,5254.8841,1.266168,5434.6644,6881.198060,6867.963397,1.034212
0.004,6171.5670,4975.7204,1.240336,5118.4467,6348.595621,6348.595621,1.028685
...,...,...,...,...,...,...,...
0.996,124.7354,144.0799,0.865738,154.9316,134.130126,144.018380,1.075317
0.997,115.2667,135.6593,0.849678,153.2661,130.226808,140.907676,1.129787
0.998,107.1446,121.0092,0.885425,152.4361,134.970770,138.409487,1.259707


## Export

In [36]:
#ngonye_synth.to_csv(output_data + 'ngonye_synthetic.csv')
#fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs.csv')
ngonye_synth.to_csv(output_data + 'ngonye_synthetic_2022.csv')
fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs_2022.csv')

In [37]:
ngonye_synth.loc[ngonye_synth.index=='1925-02-05']

,LaggedDate,VicFalls,Conversion,Flow,Exceedance
Date,,,,,
1925-02-05,1925-02-16,1238.2939,0.931312,1153.237939,0.29


In [38]:
fdc.loc[(fdc.index<=0.291) & (fdc.index>=0.288)].reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=[True,True])

,Exceedance,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion,ExceedanceInv,Ngonye_synthetic
3,0.291,1238.2939,1480.0600,1589.7696,0.778914,0.776730,1149.606712,0.928380,0.709,1153.237939
2,0.290,1238.2939,1487.2830,1598.6159,0.774604,0.775399,1153.237939,0.931312,0.710,1153.237939
1,0.289,1247.5890,1495.2593,1610.5937,0.774614,0.775074,1158.935958,0.928941,0.711,1160.715298
0,0.288,1247.5890,1497.5437,1610.5937,0.774614,0.775079,1160.715298,0.930367,0.712,1160.715298


In [39]:
vicfalls['Year']=vicfalls.index.year
vicfalls['Month']=vicfalls.index.month
vicfalls['Day']=vicfalls.index.day
vicfalls['WaterYear']=vicfalls.apply((lambda x: int(x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)

In [41]:
years=vicfalls['WaterYear'].unique().tolist()

for year in years:
    days=vicfalls.loc[vicfalls.WaterYear==year]
    days.to_csv(output_data + 'years/vicfalls_' + str(year) + '.csv')